In [ ]:
import requests
from lxml import html
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
from time import sleep
from google.colab import drive
import os
import os.path

In [ ]:
class TDCC():
    def __init__(self):
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36',
            'Host':'www.tdcc.com.tw'
        }
        self.token_url = 'https://www.tdcc.com.tw/portal/zh/smWeb/qryStock'
        self.post_url = 'https://www.tdcc.com.tw/portal/zh/smWeb/qryStock'
        self.session = requests.Session()

    def token(self, url):
        respone = self.session.get(url, headers = self.headers)
        soup = bs(respone.text , 'lxml')
        token  = soup.select('#SYNCHRONIZER_TOKEN')[0]['value']
        return token

    def getHtml(self, firDate = '20220902', scaDate = '20220902', sqlMethod = 'StockNo', stockNo = '2330'):

        post_data = {
            'SYNCHRONIZER_TOKEN': self.token(url = self.token_url),
            'SYNCHRONIZER_URI': '/portal/zh/smWeb/qryStock',
            'method': 'submit',
            'firDate': firDate,
            'scaDate': scaDate,
            'sqlMethod': sqlMethod,
            'stockNo': stockNo,
            'stockName': ''
        }
        
        response = self.session.post(self.post_url, headers= self.headers, data=post_data)
        return response.text

    def getDataFrame(self, html):
        soup = bs(html, 'lxml')
        html_table_string = str(soup.find_all("table", class_="table"))
        df_table = pd.read_html(html_table_string)
        df = df_table[0]
        return df

In [ ]:
df_all = pd.DataFrame()
date_string = '20230113'
drive.mount('/content/gdrive')

directory = f'/content/gdrive/My Drive/DataTDCCWeeklyCSV/{date_string}_CSV/'
if not os.path.exists(directory):
    os.makedirs(directory)
    sleep(5)

co_list = ["0050","2330"]
for x in range(len(co_list)):
  try:
    tdcc = TDCC()
    df = tdcc.getDataFrame(tdcc.getHtml(date_string,date_string,'StockNo',co_list[x]))
    df.insert(0,'co_id',co_list[x])
    df.insert(0,'DataDate',date_string)

    csv_file = f'{directory}/{date_string}_{co_list[x]}.csv'
    if os.path.exists(csv_file):
      print(f'\033[93m {co_list[x]} 檔案已存在 \x1b[0m')
      continue
    if len(df.index) <= 1:
      print(f'\033[93m {co_list[x]} 查無此資料 \x1b[0m')
      continue
    df.to_csv(csv_file)
    print(f'{co_list[x]} 第{x}筆 OK')
    sleep(0.3)
  except Exception as e:
      print(f'\033[91m {co_list[x]} Exception: {str(e)} \x1b[0m')

